In [30]:
from functools import partial

import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import math
import datetime

from tabulate import tabulate
from timm.models.vision_transformer import Block
# from utils import MaskEmbed, get_1d_sincos_pos_embed, ActiveEmbed, Utils

eps = 1e-6

In [31]:
import wandb

# Log in to your W&B account (if not already logged in)
wandb.login()

True

In [32]:
api = wandb.Api()

In [33]:
runs = api.runs("aneog/ett")
list_of_runs = []
for run in runs:
    list_of_runs.append((run.id, run.name))
    print(f"Run ID: {run.id}, Name: {run.name}")

Run ID: 39j04v0k, Name: finetune_v1_finetune_ETTm2_mask_75_NOT_FRZN_2024-03-07_03:57:42.461536
Run ID: t9ixni1a, Name: finetune_v1_finetune_ETTm2_mask_50_NOT_FRZN_2024-03-07_03:54:40.360778
Run ID: 3ocyl62b, Name: finetune_v1_finetune_ETTm2_mask_25_NOT_FRZN_2024-03-07_03:51:39.000794
Run ID: 1l8gm1tr, Name: finetune_v1_finetune_ETTm2_mask_75_FRZN_2024-03-07_03:48:47.458675
Run ID: 1ccpzmgl, Name: finetune_v1_finetune_ETTm2_mask_50_FRZN_2024-03-07_03:45:54.075079
Run ID: fh2jns7j, Name: finetune_v1_finetune_ETTm2_mask_25_FRZN_2024-03-07_03:43:00.682403
Run ID: 1zcu2v30, Name: finetune_v1_finetune_ETTm1_mask_75_NOT_FRZN_2024-03-07_03:32:28.937397
Run ID: 36t0ldll, Name: finetune_v1_finetune_ETTm1_mask_50_NOT_FRZN_2024-03-07_03:29:33.061686
Run ID: 19oi1mai, Name: finetune_v1_finetune_ETTm1_mask_25_NOT_FRZN_2024-03-07_03:26:41.696867
Run ID: 14isuvnn, Name: finetune_v1_finetune_ETTm1_mask_75_FRZN_2024-03-07_03:23:50.391473
Run ID: 29fcc950, Name: finetune_v1_finetune_ETTm1_mask_50_FRZN_20

In [34]:
date_regex_a = "2024-03-07"
# date_regex_b = "2024-02-21"

In [35]:
filtered_runs =  [(runid, name) for runid, name in list_of_runs if date_regex_a in name]# or date_regex_b in name]

In [36]:
filtered_runs

[('39j04v0k',
  'finetune_v1_finetune_ETTm2_mask_75_NOT_FRZN_2024-03-07_03:57:42.461536'),
 ('t9ixni1a',
  'finetune_v1_finetune_ETTm2_mask_50_NOT_FRZN_2024-03-07_03:54:40.360778'),
 ('3ocyl62b',
  'finetune_v1_finetune_ETTm2_mask_25_NOT_FRZN_2024-03-07_03:51:39.000794'),
 ('1l8gm1tr',
  'finetune_v1_finetune_ETTm2_mask_75_FRZN_2024-03-07_03:48:47.458675'),
 ('1ccpzmgl',
  'finetune_v1_finetune_ETTm2_mask_50_FRZN_2024-03-07_03:45:54.075079'),
 ('fh2jns7j',
  'finetune_v1_finetune_ETTm2_mask_25_FRZN_2024-03-07_03:43:00.682403'),
 ('1zcu2v30',
  'finetune_v1_finetune_ETTm1_mask_75_NOT_FRZN_2024-03-07_03:32:28.937397'),
 ('36t0ldll',
  'finetune_v1_finetune_ETTm1_mask_50_NOT_FRZN_2024-03-07_03:29:33.061686'),
 ('19oi1mai',
  'finetune_v1_finetune_ETTm1_mask_25_NOT_FRZN_2024-03-07_03:26:41.696867'),
 ('14isuvnn',
  'finetune_v1_finetune_ETTm1_mask_75_FRZN_2024-03-07_03:23:50.391473'),
 ('29fcc950',
  'finetune_v1_finetune_ETTm1_mask_50_FRZN_2024-03-07_03:21:05.149842'),
 ('32u9kqjr',
  'fi

In [37]:
'''
removing last couple of runs: not part of the experiments
'''
# exp_runs = filtered_runs[:-13]
exp_runs = filtered_runs

In [38]:
len(exp_runs)

36

In [39]:
exp_runs

[('39j04v0k',
  'finetune_v1_finetune_ETTm2_mask_75_NOT_FRZN_2024-03-07_03:57:42.461536'),
 ('t9ixni1a',
  'finetune_v1_finetune_ETTm2_mask_50_NOT_FRZN_2024-03-07_03:54:40.360778'),
 ('3ocyl62b',
  'finetune_v1_finetune_ETTm2_mask_25_NOT_FRZN_2024-03-07_03:51:39.000794'),
 ('1l8gm1tr',
  'finetune_v1_finetune_ETTm2_mask_75_FRZN_2024-03-07_03:48:47.458675'),
 ('1ccpzmgl',
  'finetune_v1_finetune_ETTm2_mask_50_FRZN_2024-03-07_03:45:54.075079'),
 ('fh2jns7j',
  'finetune_v1_finetune_ETTm2_mask_25_FRZN_2024-03-07_03:43:00.682403'),
 ('1zcu2v30',
  'finetune_v1_finetune_ETTm1_mask_75_NOT_FRZN_2024-03-07_03:32:28.937397'),
 ('36t0ldll',
  'finetune_v1_finetune_ETTm1_mask_50_NOT_FRZN_2024-03-07_03:29:33.061686'),
 ('19oi1mai',
  'finetune_v1_finetune_ETTm1_mask_25_NOT_FRZN_2024-03-07_03:26:41.696867'),
 ('14isuvnn',
  'finetune_v1_finetune_ETTm1_mask_75_FRZN_2024-03-07_03:23:50.391473'),
 ('29fcc950',
  'finetune_v1_finetune_ETTm1_mask_50_FRZN_2024-03-07_03:21:05.149842'),
 ('32u9kqjr',
  'fi

In [40]:
'''
sorting the list in ascending order
'''
exp_runs = exp_runs[::-1]
exp_runs

[('x19zz03e', 'pretrain_v1_pretrain_ETTh1_mask_25_2024-03-07_00:41:07.333607'),
 ('131ypcyf', 'pretrain_v1_pretrain_ETTh2_mask_25_2024-03-07_00:41:20.549567'),
 ('1wrfpgnd', 'pretrain_v1_pretrain_ETTm1_mask_25_2024-03-07_00:41:33.127734'),
 ('2vnyaum4', 'pretrain_v1_pretrain_ETTm2_mask_25_2024-03-07_00:41:49.832274'),
 ('2g69x0nu', 'pretrain_v1_pretrain_ETTh2_mask_50_2024-03-07_00:57:08.839516'),
 ('2t7mg726', 'pretrain_v1_pretrain_ETTh1_mask_50_2024-03-07_01:02:00.496959'),
 ('1nyz1ixk', 'pretrain_v1_pretrain_ETTh2_mask_75_2024-03-07_01:14:42.780737'),
 ('1twcqzl1',
  'finetune_v1_finetune_ETTh2_mask_25_FRZN_2024-03-07_01:30:38.955004'),
 ('292ze8lq', 'pretrain_v1_pretrain_ETTh1_mask_75_2024-03-07_01:32:05.357228'),
 ('1myxbckx', 'pretrain_v1_pretrain_ETTm1_mask_50_2024-03-07_01:35:05.214247'),
 ('35k9ru3r',
  'finetune_v1_finetune_ETTh2_mask_50_FRZN_2024-03-07_01:35:15.872944'),
 ('zdy9t75w',
  'finetune_v1_finetune_ETTh2_mask_75_FRZN_2024-03-07_01:39:55.592154'),
 ('3gjh7sjx',
  'fi

In [57]:
summaries = []

pretrain_val_features = ['val_mse', 'val_mae']
pretrain_test_features = ['test_mse', 'test_mae']

finetune_val_features = ['val_mse']
finetune_train_features = ['train_mse', 'train_mae']
finetune_test_features = ['test_mse', 'test_mae']

In [45]:
'''
using the api, get all the values for the corresponding experiments
'''

'\nusing the api, get all the values for the corresponding experiments\n'

In [54]:
'''
all of these runs are in order (sorted wrt time)
'''
etth1 = [(runid, name) for runid, name in exp_runs if 'ETTh1' in name]

etth2 = [(runid, name) for runid, name in exp_runs if 'ETTh2' in name]

ettm1 = [(runid, name) for runid, name in exp_runs if 'ETTm1' in name]

ettm2 = [(runid, name) for runid, name in exp_runs if 'ETTm2' in name]

In [47]:
etth1

[('x19zz03e', 'pretrain_v1_pretrain_ETTh1_mask_25_2024-03-07_00:41:07.333607'),
 ('2t7mg726', 'pretrain_v1_pretrain_ETTh1_mask_50_2024-03-07_01:02:00.496959'),
 ('292ze8lq', 'pretrain_v1_pretrain_ETTh1_mask_75_2024-03-07_01:32:05.357228'),
 ('1v2gp3dn',
  'finetune_v1_finetune_ETTh1_mask_25_FRZN_2024-03-07_01:57:05.598705'),
 ('3uty5ien',
  'finetune_v1_finetune_ETTh1_mask_50_FRZN_2024-03-07_02:03:03.896646'),
 ('3ho2pzil',
  'finetune_v1_finetune_ETTh1_mask_75_FRZN_2024-03-07_02:07:29.426500'),
 ('2wyj0uj4',
  'finetune_v1_finetune_ETTh1_mask_25_NOT_FRZN_2024-03-07_02:11:35.182923'),
 ('xij5g50n',
  'finetune_v1_finetune_ETTh1_mask_50_NOT_FRZN_2024-03-07_02:15:25.089525'),
 ('2219pfsh',
  'finetune_v1_finetune_ETTh1_mask_75_NOT_FRZN_2024-03-07_02:19:16.772993')]

In [48]:
etth2

[('131ypcyf', 'pretrain_v1_pretrain_ETTh2_mask_25_2024-03-07_00:41:20.549567'),
 ('2g69x0nu', 'pretrain_v1_pretrain_ETTh2_mask_50_2024-03-07_00:57:08.839516'),
 ('1nyz1ixk', 'pretrain_v1_pretrain_ETTh2_mask_75_2024-03-07_01:14:42.780737'),
 ('1twcqzl1',
  'finetune_v1_finetune_ETTh2_mask_25_FRZN_2024-03-07_01:30:38.955004'),
 ('35k9ru3r',
  'finetune_v1_finetune_ETTh2_mask_50_FRZN_2024-03-07_01:35:15.872944'),
 ('zdy9t75w',
  'finetune_v1_finetune_ETTh2_mask_75_FRZN_2024-03-07_01:39:55.592154'),
 ('3gjh7sjx',
  'finetune_v1_finetune_ETTh2_mask_25_NOT_FRZN_2024-03-07_01:44:24.349852'),
 ('lrarbatd',
  'finetune_v1_finetune_ETTh2_mask_50_NOT_FRZN_2024-03-07_01:49:00.393258'),
 ('1z7zhbdv',
  'finetune_v1_finetune_ETTh2_mask_75_NOT_FRZN_2024-03-07_01:53:34.213236')]

In [49]:
ettm1

[('1wrfpgnd', 'pretrain_v1_pretrain_ETTm1_mask_25_2024-03-07_00:41:33.127734'),
 ('1myxbckx', 'pretrain_v1_pretrain_ETTm1_mask_50_2024-03-07_01:35:05.214247'),
 ('3ngoqhxm', 'pretrain_v1_pretrain_ETTm1_mask_75_2024-03-07_02:25:46.369279'),
 ('32u9kqjr',
  'finetune_v1_finetune_ETTm1_mask_25_FRZN_2024-03-07_03:18:21.814298'),
 ('29fcc950',
  'finetune_v1_finetune_ETTm1_mask_50_FRZN_2024-03-07_03:21:05.149842'),
 ('14isuvnn',
  'finetune_v1_finetune_ETTm1_mask_75_FRZN_2024-03-07_03:23:50.391473'),
 ('19oi1mai',
  'finetune_v1_finetune_ETTm1_mask_25_NOT_FRZN_2024-03-07_03:26:41.696867'),
 ('36t0ldll',
  'finetune_v1_finetune_ETTm1_mask_50_NOT_FRZN_2024-03-07_03:29:33.061686'),
 ('1zcu2v30',
  'finetune_v1_finetune_ETTm1_mask_75_NOT_FRZN_2024-03-07_03:32:28.937397')]

In [50]:
ettm2

[('2vnyaum4', 'pretrain_v1_pretrain_ETTm2_mask_25_2024-03-07_00:41:49.832274'),
 ('lr2a30rp', 'pretrain_v1_pretrain_ETTm2_mask_50_2024-03-07_01:51:52.357977'),
 ('z6c9nkn5', 'pretrain_v1_pretrain_ETTm2_mask_75_2024-03-07_02:49:10.389465'),
 ('fh2jns7j',
  'finetune_v1_finetune_ETTm2_mask_25_FRZN_2024-03-07_03:43:00.682403'),
 ('1ccpzmgl',
  'finetune_v1_finetune_ETTm2_mask_50_FRZN_2024-03-07_03:45:54.075079'),
 ('1l8gm1tr',
  'finetune_v1_finetune_ETTm2_mask_75_FRZN_2024-03-07_03:48:47.458675'),
 ('3ocyl62b',
  'finetune_v1_finetune_ETTm2_mask_25_NOT_FRZN_2024-03-07_03:51:39.000794'),
 ('t9ixni1a',
  'finetune_v1_finetune_ETTm2_mask_50_NOT_FRZN_2024-03-07_03:54:40.360778'),
 ('39j04v0k',
  'finetune_v1_finetune_ETTm2_mask_75_NOT_FRZN_2024-03-07_03:57:42.461536')]

In [51]:
finetune_cols = ['Finetune25_Frozen |', '| Finetune50_Frozen |', '| Finetune75_Frozen |', '| Finetune25_NF |', '| Finetune50_NF |', '| Finetune75_NF |']
pretrain_cols = ['Pretrain_25 |', '| Pretrain_50 |', '| Pretrain_75 |']

In [59]:
def highlight_min(s):
    is_min = s == s.min()
    # styles = ['background-color: yellow' if v else '' for v in is_min]
    text_color = ['color: red' if v else '' for v in is_min]
    return text_color

def dataset_runs(dataset_ls):
    summaries_pretrain = []
    summaries_finetune = []
    
    for run_id, name in dataset_ls:
        run = api.run(f"aneog/ett/{run_id}")

        summary = run.summary
        params = run.config

        if 'finetune' in name:
            print(f"Summary for Run {name}:")
            summary_dict_val = {}
            summary_dict_train = {}
            summary_dict_test = {}
            
            # for key_val in finetune_val_features:
            #     if key_val in summary:
            #         summary_dict_val[key_val] = summary[key_val]
            #     else:
            #         print(f"Finetune: Key: {key_val} not found in the dictionary.")
            
            for key_train in finetune_train_features:
                if key_train in summary:
                    summary_dict_train[key_train] = summary[key_train]
                else:
                    print(f"Finetune: Key: {key_train} not found in the dictionary.")
                    
            for key_test in finetune_test_features:
                if key_test in summary:
                    summary_dict_test[key_test] = summary[key_test]
                else:
                    print(f"Finetune: Key: {key_test} not found in the dictionary.")
                    
            summaries_finetune.append(list(summary_dict_test.values()) + list(summary_dict_train.values()))
            
        if 'pretrain' in name:
            print(f"Summary for Run {name}:")
            summary_val_dict = {}
            summary_test_dict = {}
            
            for key in pretrain_val_features:
                if key in summary:
                    summary_val_dict[key] = summary[key]
                else:
                    print(f"Pretrain: Key: {key} not found in the dictionary.")
                    
            for key in pretrain_test_features:
                if key in summary:
                    summary_test_dict[key] = summary[key]
                else:
                    print(f"Pretrain: Key: {key} not found in the dictionary.")
                    
            summaries_pretrain.append(list(summary_test_dict.values()) + list(summary_val_dict.values()))
    
    if len(summaries_finetune)!=0:
        finetune_df = pd.DataFrame(summaries_finetune, columns=finetune_test_features + finetune_train_features).transpose()
        finetune_df.columns = finetune_cols
    
    if len(summaries_pretrain)!=0:
        pretrain_df = pd.DataFrame(summaries_pretrain, columns=pretrain_test_features + pretrain_val_features).transpose()
        pretrain_df.columns = pretrain_cols
        
    if len(summaries_pretrain)!=0 and len(summaries_finetune)!=0:
        return pretrain_df.style.apply(highlight_min, axis=1), finetune_df.style.apply(highlight_min, axis=1)
    elif len(summaries_pretrain)!=0:
        return pretrain_df.style.apply(highlight_min, axis=1), None
    else:
        return None, finetune_df.style.apply(highlight_min, axis=1)

In [60]:
etth1_pretrain, etth1_finetune = dataset_runs(etth1)

Summary for Run pretrain_v1_pretrain_ETTh1_mask_25_2024-03-07_00:41:07.333607:
Summary for Run pretrain_v1_pretrain_ETTh1_mask_50_2024-03-07_01:02:00.496959:
Summary for Run pretrain_v1_pretrain_ETTh1_mask_75_2024-03-07_01:32:05.357228:
Summary for Run finetune_v1_finetune_ETTh1_mask_25_FRZN_2024-03-07_01:57:05.598705:
Summary for Run finetune_v1_finetune_ETTh1_mask_50_FRZN_2024-03-07_02:03:03.896646:
Summary for Run finetune_v1_finetune_ETTh1_mask_75_FRZN_2024-03-07_02:07:29.426500:
Summary for Run finetune_v1_finetune_ETTh1_mask_25_NOT_FRZN_2024-03-07_02:11:35.182923:
Summary for Run finetune_v1_finetune_ETTh1_mask_50_NOT_FRZN_2024-03-07_02:15:25.089525:
Summary for Run finetune_v1_finetune_ETTh1_mask_75_NOT_FRZN_2024-03-07_02:19:16.772993:


In [61]:
etth2_pretrain, etth2_finetune = dataset_runs(etth2)

Summary for Run pretrain_v1_pretrain_ETTh2_mask_25_2024-03-07_00:41:20.549567:
Summary for Run pretrain_v1_pretrain_ETTh2_mask_50_2024-03-07_00:57:08.839516:
Summary for Run pretrain_v1_pretrain_ETTh2_mask_75_2024-03-07_01:14:42.780737:
Summary for Run finetune_v1_finetune_ETTh2_mask_25_FRZN_2024-03-07_01:30:38.955004:
Summary for Run finetune_v1_finetune_ETTh2_mask_50_FRZN_2024-03-07_01:35:15.872944:
Summary for Run finetune_v1_finetune_ETTh2_mask_75_FRZN_2024-03-07_01:39:55.592154:
Summary for Run finetune_v1_finetune_ETTh2_mask_25_NOT_FRZN_2024-03-07_01:44:24.349852:
Summary for Run finetune_v1_finetune_ETTh2_mask_50_NOT_FRZN_2024-03-07_01:49:00.393258:
Summary for Run finetune_v1_finetune_ETTh2_mask_75_NOT_FRZN_2024-03-07_01:53:34.213236:


In [62]:
ettm1_pretrain, ettm1_finetune = dataset_runs(ettm1)

Summary for Run pretrain_v1_pretrain_ETTm1_mask_25_2024-03-07_00:41:33.127734:
Summary for Run pretrain_v1_pretrain_ETTm1_mask_50_2024-03-07_01:35:05.214247:
Summary for Run pretrain_v1_pretrain_ETTm1_mask_75_2024-03-07_02:25:46.369279:
Summary for Run finetune_v1_finetune_ETTm1_mask_25_FRZN_2024-03-07_03:18:21.814298:
Summary for Run finetune_v1_finetune_ETTm1_mask_50_FRZN_2024-03-07_03:21:05.149842:
Summary for Run finetune_v1_finetune_ETTm1_mask_75_FRZN_2024-03-07_03:23:50.391473:
Summary for Run finetune_v1_finetune_ETTm1_mask_25_NOT_FRZN_2024-03-07_03:26:41.696867:
Summary for Run finetune_v1_finetune_ETTm1_mask_50_NOT_FRZN_2024-03-07_03:29:33.061686:
Summary for Run finetune_v1_finetune_ETTm1_mask_75_NOT_FRZN_2024-03-07_03:32:28.937397:


In [63]:
ettm2_pretrain, ettm2_finetune = dataset_runs(ettm2)

Summary for Run pretrain_v1_pretrain_ETTm2_mask_25_2024-03-07_00:41:49.832274:
Summary for Run pretrain_v1_pretrain_ETTm2_mask_50_2024-03-07_01:51:52.357977:
Summary for Run pretrain_v1_pretrain_ETTm2_mask_75_2024-03-07_02:49:10.389465:
Summary for Run finetune_v1_finetune_ETTm2_mask_25_FRZN_2024-03-07_03:43:00.682403:
Summary for Run finetune_v1_finetune_ETTm2_mask_50_FRZN_2024-03-07_03:45:54.075079:
Summary for Run finetune_v1_finetune_ETTm2_mask_75_FRZN_2024-03-07_03:48:47.458675:
Summary for Run finetune_v1_finetune_ETTm2_mask_25_NOT_FRZN_2024-03-07_03:51:39.000794:
Summary for Run finetune_v1_finetune_ETTm2_mask_50_NOT_FRZN_2024-03-07_03:54:40.360778:
Summary for Run finetune_v1_finetune_ETTm2_mask_75_NOT_FRZN_2024-03-07_03:57:42.461536:


In [65]:
legend = """
    <h3>Legend:</h3>
    <ul>
        <li>Red text: Lowest MSE and MAE value in each row</li>
    </ul>
    <ul>
        <li>Finetune25_FROZEN: 25% Masked Pretrained model finetuned with a Frozen encoder
    </ul>
    <ul>
        <li>Finetune75_NF: 75% Masked Pretrained model Fully Fine-Tuned (i.e. encoder not frozen)
    </ul>
    <br>
    <h3>Training Procedure:</h3>
    <ul>
        <li> Data was split in the ratio = 6:2:2 (standard for ETT)
        <li> Lookback Window = 336 timesteps
        <li> Horizon Window = 96 timesteps
        <li> Pre-trained epochs = 50
        <li> Fine-tune epochs = 10
        <li> 3 Models were pretrained on 60% of each dataset with following mask ratios: 25, 50, 75
        <li> Once pretrained, each model was finetuned on each dataset (60% portion of it) and tested on the 20%
    </ul>
    """

with open('ett_results_updated.html', 'w') as f:
    f.write('<html><body>')
    f.write(legend)
    # f.write('<h2> ETTh1 Pre-train </h2>')
    # f.write(combined_pretrain.render())
    
    f.write('<h2> ETTh1 </h2>')
    f.write(etth1_finetune.render())
    
    f.write('<h2> ETTh2 </h2>')
    f.write(etth2_finetune.render())
    
    f.write('<h2> ETTm1 </h2>')
    f.write(ettm1_finetune.render())
    
    f.write('<h2> ETTm2 </h2>')
    f.write(ettm2_finetune.render())
    
#     f.write('<h2> PRLA Observed Downstream </h2>')
#     f.write(prla_obs_finetune.render())
    
#     f.write('<h2> PRPO Observed Downstream </h2>')
#     f.write(prpo_obs_finetune.render())
    
#     f.write('<h2> SUGG Observed Downstream </h2>')
#     f.write(sugg_obs_finetune.render())
    
#     f.write('<h2> TOOK Observed Downstream </h2>')
#     f.write(took_obs_finetune.render())
    
    
    f.write('</body></html>')

/tmp/ipykernel_30188/3679645044.py:32: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(etth1_finetune.render())
/tmp/ipykernel_30188/3679645044.py:35: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(etth2_finetune.render())
/tmp/ipykernel_30188/3679645044.py:38: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(ettm1_finetune.render())
/tmp/ipykernel_30188/3679645044.py:41: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(ettm2_finetune.render())
